# Records and Protocols

### Striking a More Specific Bargain with Records


In [2]:
;; Records are defined as follows

(defrecord FictionalCharacter[name appears-in author])


user.FictionalCharacter

In [5]:
;; The standard instantiation syntax using '->FictionalCharacter'

(def watson 
    (->FictionalCharacter "John Watson" "Sign of the Four" "Doyle")) ; Individual strings are passed as args

watson

#user.FictionalCharacter{:name "John Watson", :appears-in "Sign of the Four", :author "Doyle"}

In [6]:
;; An alternative instantiation syntax using 'map->FictionalCharacter'

(def elizabeth 
    (map->FictionalCharacter {:name "Elizabeth Bennet"
                              :appears-in "Pride & Prejudice"
                              :author "Austen"})) ; A map is passed as arg

elizabeth

#user.FictionalCharacter{:name "Elizabeth Bennet", :appears-in "Pride & Prejudice", :author "Austen"}

### Records are maps

In [19]:
;; Records can be treated like maps with keyword keys

(:name elizabeth)


"Elizabeth Bennet"

In [16]:
(:appears-in watson)


"Sign of the Four"

In [14]:
;; Any function that works with a map will also work with a record

(count elizabeth)


3

In [15]:
(keys watson)


(:name :appears-in :author)

In [18]:
;; You can use 'assoc' to modify the values in a record

(def specific-watson 
    (assoc watson :appears-in "Sign of the Four"))


#'user/specific-watson

In [21]:
;; You can use 'assoc' to add a new key-value par in a record

(def more-about-watson 
    (assoc watson :address "221B Baker Street")) ; New keys don't get the speed boost of the built-in fields


#'user/more-about-watson

### The record advantage

In [22]:
;; Records are faster than maps

(def irene {:name "Irene Adler"
            :appears-in "A Scandal in Bohemia"
            :author "Doyle"})


#'user/irene

In [24]:
;; So this would be faster:

(:name watson)


"John Watson"

In [23]:
; Than this:

(:name irene)


"Irene Adler"

In [25]:
;; Records can make code more clearer

(defrecord FictionalCharacter[name appears-in author])

(defrecord SuperComputer [cpu no-cpus storage-gb])



user.SuperComputer

In [28]:
;; So when instantiatong them, you can have an idea of what each record is about
;; by looking at their definitions

(def watson-1 (->FictionalCharacter "John Watson" "Sign of the Four" "Doyle")) ; It's a fictional character

watson-1

#user.FictionalCharacter{:name "John Watson", :appears-in "Sign of the Four", :author "Doyle"}

In [29]:
(def watson-2 (->SuperComputer "Power7" 2880 4000)) ; It's the supercomputer that runs Jeopardy!

watson-2

#user.SuperComputer{:cpu "Power7", :no-cpus 2880, :storage-gb 4000}

In [30]:
;; Like in OOP languages, you can check the class of each record instance

(class watson-1)


user.FictionalCharacter

In [31]:
(class watson-2)

user.SuperComputer

In [32]:
;; Or check if a record belongs to some class

(instance? FictionalCharacter watson-1)


true

In [34]:
(instance? SuperComputer watson-1)


false

### Protocols

In [37]:
;; A simple protocol 'Person' with some functions related to people

(defprotocol Person
    (full-name [this])
    (greeting [this msg])
    (description [this]))


Person

In [40]:
;; The 'FictionalCharacter' record defined before can implement the functions
;; of the 'Person' protocol, as follows

(defrecord FictionalCharacter[name appears-in author]
    Person
    (full-name [this] 
        (:name this))
    (greeting [this msg] 
        (str msg " " (:name this)))
    (description [this]
        (str (:name this) " is a character in " (:appears-in this))))


user.FictionalCharacter

In [39]:
;; Another record 'Employee' can also implement the same functions 
;; of the 'Person' protocol (differently), as follows


(defrecord Employee [first-name last-name department]
    Person
    (full-name [this] 
        (str first-name " " last-name))
    (greeting [this msg] 
        (str msg " " (:first-name this)))
    (description [this]
        (str (:first-name this) " works in " (:department this))))


user.Employee

In [42]:
;; Now we can make instances of both records

(def sofia (->Employee "Sofia" "Diego" "Finance"))

(def sam (->FictionalCharacter "Sam Weller" "The Pickwick Papers" "Dickens"))

#'user/sam

In [43]:
;; And we can see the polymorphism of protocols in action, as follows

(full-name sofia)


"Sofia Diego"

In [44]:
(full-name sam)

"Sam Weller"

In [46]:
(description sofia)


"Sofia works in Finance"

In [45]:
(description sam)


"Sam Weller is a character in The Pickwick Papers"

### Decentralized Polymorphism


In [48]:
;; A new simple protocol

(defprotocol Marketable
    (make-slogan [this]))


Marketable

In [49]:
;; To apply the new protocol to existing records, without modifying the original definitions
;; the 'extend-protocol' macro can be used, as follows

(extend-protocol Marketable
    Employee
    (make-slogan [e] 
        (str (:first-name e) " is the BEST employee!"))
    FictionalCharacter
    (make-slogan [fc] 
        (str (:name fc) " is the GREATEST character!"))
    SuperComputer
    (make-slogan [sc] 
        (str "This computer has " (:no-cpus sc) " CPUs!")))


nil

In [50]:
;; You can extend the new protocol to embrace data types that aren’t records, as follows

(extend-protocol Marketable
    String
    (make-slogan [s] 
        (str \" s \" " is a string! WOW!"))
    Boolean
    (make-slogan [b] 
        (str b " is one of the two surviving Booleans!")))


nil

In [51]:
;; Given the following protocol from the component library (https://github.com/stuartsierra/component)

(defprotocol Lifecycle
    (start [component]
        "Begins operation of this component. Synchronous, does not return
        until the component is started. Returns an updated version of this
        component.")
    (stop [component]
        "Ceases operation of this component. Synchronous, does not return
        until the component is stopped. Returns an updated version of this
        component."))

; You can create a one-off implementation of that protocol with 'reify', as follows

(def test-component 
    (reify Lifecycle
        (start [this]
            (println "Start!") this)
        (stop [this]
            (println "Stop!") this)))


#'user/test-component

### Issues with records and protocols

In [53]:
;; Instantiating a record with keys that are not present in the definition
;; will add them to the record, but they will be treated as map keys

(map->FictionalCharacter {:full-name "Elizabeth Bennet"
                          :book "Pride & Prejudice"
                          :written-by "Austen"})


#user.FictionalCharacter{:name nil, :appears-in nil, :author nil, :full-name "Elizabeth Bennet", :book "Pride & Prejudice", :written-by "Austen"}

In [55]:
;; The same can happen with carelles use of 'assoc'

(assoc elizabeth :book "Pride & Prejudice")


#user.FictionalCharacter{:name "Elizabeth Bennet", :appears-in "Pride & Prejudice", :author "Austen", :book "Pride & Prejudice"}

In [56]:
;; Be careful with protocols with the same method names

(defprotocol Person
    (full-name [this])
    (greeting [this msg])
    (description [this]))

(defprotocol Product
    (inventory-name [this])
    (description [this])) ; This method will override the one in 'Person'



Product

In [60]:
;; The solution is simple: put each protocol in a separate namespace

(ns person-ns)
(defprotocol Person
    (full-name [this])
    (greeting [this msg])
    (description [this]))

(ns product-ns)
(defprotocol Product
    (inventory-name [this])
    (description [this])) ; This method will override the one in 'Person'

(ns user)

nil

nil